In [2]:
# load train dataset
import pandas as pd
df_train = pd.read_csv('data/Dataset for participants V2/train.csv')
df_test = pd.read_csv('data/Dataset for participants V2/test.csv')
df_train.head()

,tag_id,image_id,p1_x,p_1y,p2_x,p2_y,p3_x,p3_y,p4_x,p4_y,...,enclosed_cab,spare_wheel,wrecked,flatbed,ladder,enclosed_box,soft_shell_box,harnessed_to_a_cart,ac_vents,color
0,15036,16490,3988.399902,1227.588745,3986.710693,1213.288208,4002.600098,1211.411255,4004.289307,1225.711792,...,0,0,0,0,0,0,0,0,-1,black
1,31658,16490,3380.804152,33.370945,3329.811654,128.465062,3358.064254,142.935906,3409.056752,44.396350,...,-1,-1,0,-1,0,1,0,0,-1,red
2,26971,17122,3948.334473,1426.797241,3958.876221,1410.750122,4000.665527,1438.202759,3990.123779,1454.249878,...,0,0,0,-1,-1,-1,-1,-1,-1,red
3,19301,12193,1977.798007,1009.010733,1983.435998,992.096760,2034.177918,1009.010733,2027.913483,1027.177593,...,0,0,0,-1,-1,-1,-1,-1,-1,silver/grey
4,35906,12193,3196.620177,1284.240545,3193.174738,1311.804057,3288.957942,1321.451286,3292.403381,1293.198686,...,-1,-1,0,1,0,0,0,0,-1,red


In [22]:
# Prepare vehicles dataset
import math,os,fnmatch
from vehicle_crop import *
from PIL import Image

import vehicle_crop
import importlib
importlib.reload(vehicle_crop)

def prepare_images(df, in_dir, out_dir, is_training_data):
    images_fname = os.listdir(in_dir)
    last_image_id = None
    for index, row in df.iterrows():
        image_id = int(row['image_id'])
        if (image_id != last_image_id):
            fname = fnmatch.filter(images_fname, '{}.*'.format(image_id))[0]
            im = Image.open('{}/{}'.format(in_dir, fname))
            width, height = im.size

        points = [
            (row['p1_x'],row['p_1y']),
            (row[' p2_x'],row[' p2_y']),
            (row[' p3_x'],row[' p3_y']),
            (row[' p4_x'],row[' p4_y'])
        ]

        classes = ['general_class','sub_class','sunroof','luggage_carrier','open_cargo_area','enclosed_cab','spare_wheel','wrecked','flatbed','ladder','enclosed_box','soft_shell_box','harnessed_to_a_cart','ac_vents']
        cropped = crop_vehicle(im, points, 15)
        processed_im = make_square(cropped, 10)
        # Comment this to generate all dataset
    #     processed_im.show(); break
    
        for clazz in classes:
            save_class_images(clazz, row, index, processed_im, out_dir, is_training_data)
            
        no_margins = crop_vehicle(im, points, 0)
        save_class_images('color', row, index, no_margins, out_dir, is_training_data)

def save_class_images(clazz, row, index, im, out_dir, is_training_data):
    tag_id = int(row['tag_id'])
    if is_training_data:
        split = 'validation' if index%5==0 else 'train'
        out_dir = '{}/{}/{}/{}'.format(out_dir,clazz,split,row[clazz])
    else:
        out_dir = '{}/{}/test/all'.format(out_dir,clazz)
        
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    im.save('{}/{}.bmp'.format(out_dir,tag_id))

def make_square(im, min_size=256, fill_color=(0, 0, 0, 0)):
    x, y = im.size
    size = max(min_size, x, y)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    return new_im


prepare_images(df_train,'data/Dataset for participants V2/training imagery', 'data/train', True)
prepare_images(df_test,'data/Dataset for participants V2/test imagery', 'data/test', False)
print('Done!')

Done!


In [16]:
# Show specific car (aligned, but without margins)
import fnmatch
from PIL import Image
from vehicle_crop import *
from vehicle_crop.diagonal_crop.point import *

import vehicle_crop
import importlib
importlib.reload(vehicle_crop)

row = df_train.loc[df_train['tag_id'] == 22907].iloc[0]
image_id = row['image_id']
fname = fnmatch.filter(images_fname, str(image_id) + '.*')[0]
im = Image.open('data/Dataset for participants V2/training imagery/' + fname)

points = [
    Point(row['p1_x'],row['p_1y']),
    Point(row[' p2_x'],row[' p2_y']),
    Point(row[' p3_x'],row[' p3_y']),
    Point(row[' p4_x'],row[' p4_y'])
]

bounds = getBounds(points)
im.crop((bounds.left, bounds.upper, bounds.right, bounds.lower)).show()

crop_vehicle(im, points, 15).show()